#### https://mp.weixin.qq.com/s/QhYqhC-eMuuXTlkNvCIhcA

本文介绍了Pandas库中处理时间序列数据的几种常用方法。

在时间格式转换部分，介绍了两种将时间转化成日期类型的方法，分别是通过设置参数parse_dates和调用方法pd.to_datetime()。

接着，介绍了时间周期的转换，通过调用.resample()方法实现，包括降采样和升采样。

最后，介绍两个常用的窗口函数rolling和expanding。

# 1、时间格式转换
介绍两种方法。第一种方法是用pandas.read_csv导入文件的时候，通过设置参数parse_dates和index_col，直接对日期列进行转换，并将其设置为索引。

In [ ]:
data = pd.read_csv('unemployment.csv', parse_dates=['date'], index_col='date')

第二种方法是在已经导入数据的情况下，用pd.to_datetime()【2】将列转换成日期类型，再用 df.set_index()【3】将其设置为索引，完成转换。

In [1]:
import tushare as ts
import pandas as pd

pd.set_option('expand_frame_repr', False)  # 列太多时不换行
pro = ts.pro_api()

df = pro.daily(ts_code='000001.SZ', start_date='20180701', end_date='20180718')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 11 columns):
ts_code       13 non-null object
trade_date    13 non-null object
open          13 non-null float64
high          13 non-null float64
low           13 non-null float64
close         13 non-null float64
pre_close     13 non-null float64
change        13 non-null float64
pct_chg       13 non-null float64
vol           13 non-null float64
amount        13 non-null float64
dtypes: float64(9), object(2)
memory usage: 1.2+ KB


In [2]:
df.head()

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,000001.SZ,20180718,8.75,8.85,8.69,8.70,8.72,-0.02,-0.23,525152.77,460697.377
1,000001.SZ,20180717,8.74,8.75,8.66,8.72,8.73,-0.01,-0.11,375356.33,326396.994
2,000001.SZ,20180716,8.85,8.90,8.69,8.73,8.88,-0.15,-1.69,689845.58,603427.713
3,000001.SZ,20180713,8.92,8.94,8.82,8.88,8.88,0.00,0.00,603378.21,535401.175
4,000001.SZ,20180712,8.60,8.97,8.58,8.88,8.64,0.24,2.78,1140492.31,1008658.828


In [2]:
df['trade_date'] = pd.to_datetime(df['trade_date'])
df.set_index('trade_date', inplace=True)
df.sort_values('trade_date', ascending=True, inplace=True)  # 升序排列

df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 13 entries, 2018-07-02 to 2018-07-18
Data columns (total 10 columns):
ts_code      13 non-null object
open         13 non-null float64
high         13 non-null float64
low          13 non-null float64
close        13 non-null float64
pre_close    13 non-null float64
change       13 non-null float64
pct_chg      13 non-null float64
vol          13 non-null float64
amount       13 non-null float64
dtypes: float64(9), object(1)
memory usage: 1.1+ KB


In [3]:
df.head()

,ts_code,open,high,low,close,pre_close,change,pct_chg,vol,amount
trade_date,,,,,,,,,,
2018-07-02,000001.SZ,9.05,9.05,8.55,8.61,9.09,-0.48,-5.28,1315520.13,1158545.868
2018-07-03,000001.SZ,8.69,8.70,8.45,8.67,8.61,0.06,0.70,1274838.57,1096657.033
2018-07-04,000001.SZ,8.63,8.75,8.61,8.61,8.67,-0.06,-0.69,711153.37,617278.559
2018-07-05,000001.SZ,8.62,8.73,8.55,8.60,8.61,-0.01,-0.12,835768.77,722169.579
2018-07-06,000001.SZ,8.61,8.78,8.45,8.66,8.60,0.06,0.70,988282.69,852071.526


# 2、时间周期转换
重采样指的是将时间序列从⼀个频率转换到另⼀个频率的处理过程。将⾼频率数据聚合到低频率称为降采样downsampling，如将股票的日线数据转换成周线数据，⽽将低频率数据转换到⾼频率则称为升采样upsampling，如将股票的周线数据转换成日线数据。

转换的思路是这样的，以日历中的周进行聚合，如'20180702'-'20180708'，取该周期内，日线开盘价的第一个值作为周开盘价，日线最高价的最大值作为周最高价，日线最低价的最小值作为周最低价，日线收盘价的最后一个值作为周最收盘价，日线最高价的最大值作为周最高价，日线成交量的求和作为周成交量（手），如下图黄色方框所示。

## 降采样：以日线数据转换周线数据为例。

In [4]:
freq = '1W'
df_weekly = df[['open']].resample(rule=freq).first()
df_weekly['high'] = df['high'].resample(rule=freq).max()
df_weekly['low'] = df['low'].resample(rule=freq).min()
df_weekly['close'] = df['close'].resample(rule=freq).last()
df_weekly['vol'] = df['vol'].resample(rule=freq).sum()

df_weekly

,open,high,low,close,vol
trade_date,,,,,
2018-07-08,9.05,9.05,8.45,8.66,5125563.53
2018-07-15,8.69,9.03,8.58,8.88,4901983.84
2018-07-22,8.85,8.90,8.66,8.70,1590354.68


## 升采样：以周线数据转换日线数据为例。继续使用上面刚刚转换好的周线数据，我们再试着把它转换成日线数据。

先通过.resample('D').asfreq()【5】方法，将周线数据的频率转换成日线，

In [5]:
df_daily = df_weekly.resample('D').asfreq()
df_daily

,open,high,low,close,vol
trade_date,,,,,
2018-07-08,9.05,9.05,8.45,8.66,5125563.53
2018-07-09,NaN,NaN,NaN,NaN,NaN
2018-07-10,NaN,NaN,NaN,NaN,NaN
2018-07-11,NaN,NaN,NaN,NaN,NaN
2018-07-12,NaN,NaN,NaN,NaN,NaN
2018-07-13,NaN,NaN,NaN,NaN,NaN
2018-07-14,NaN,NaN,NaN,NaN,NaN
2018-07-15,8.69,9.03,8.58,8.88,4901983.84
2018-07-16,NaN,NaN,NaN,NaN,NaN


.ffill()代表用前值进行填充，也就是用前面的非空值对后面的NaN值进行填充

In [6]:
df_daily = df_weekly.resample('D').ffill()
df_daily

,open,high,low,close,vol
trade_date,,,,,
2018-07-08,9.05,9.05,8.45,8.66,5125563.53
2018-07-09,9.05,9.05,8.45,8.66,5125563.53
2018-07-10,9.05,9.05,8.45,8.66,5125563.53
2018-07-11,9.05,9.05,8.45,8.66,5125563.53
2018-07-12,9.05,9.05,8.45,8.66,5125563.53
2018-07-13,9.05,9.05,8.45,8.66,5125563.53
2018-07-14,9.05,9.05,8.45,8.66,5125563.53
2018-07-15,8.69,9.03,8.58,8.88,4901983.84
2018-07-16,8.69,9.03,8.58,8.88,4901983.84


同理，.bfill()代表用后值对空值进行填充，效果如下。

In [7]:
df_daily = df_weekly.resample('D').bfill()
df_daily

,open,high,low,close,vol
trade_date,,,,,
2018-07-08,9.05,9.05,8.45,8.66,5125563.53
2018-07-09,8.69,9.03,8.58,8.88,4901983.84
2018-07-10,8.69,9.03,8.58,8.88,4901983.84
2018-07-11,8.69,9.03,8.58,8.88,4901983.84
2018-07-12,8.69,9.03,8.58,8.88,4901983.84
2018-07-13,8.69,9.03,8.58,8.88,4901983.84
2018-07-14,8.69,9.03,8.58,8.88,4901983.84
2018-07-15,8.69,9.03,8.58,8.88,4901983.84
2018-07-16,8.85,8.90,8.66,8.70,1590354.68


# 3、时间窗口函数
当我们想要比较数据在相同时间窗口的不同特征和变化时，可以借助窗口函数rolling【6】进行计算。

看一个实例：计算股票收盘价的移动平均值。

In [9]:
df = df[['ts_code', 'close']]
df

,ts_code,close
trade_date,,
2018-07-02,000001.SZ,8.61
2018-07-03,000001.SZ,8.67
2018-07-04,000001.SZ,8.61
2018-07-05,000001.SZ,8.60
2018-07-06,000001.SZ,8.66
2018-07-09,000001.SZ,9.03
2018-07-10,000001.SZ,8.98
2018-07-11,000001.SZ,8.78
2018-07-12,000001.SZ,8.88


调用rolling函数，通过设置参数window的值规定窗口大小，这里设置为3，并且调用.mean()方法计算窗口期为3天的均值，结果如下。

其中，'20180704'当天的平均值等于'20180702'-'20180704'三天的收盘价取平均的结果，'20180705'当天的平均值等于'20180703'-'20180705'三天的收盘价取平均的结果，以此类推。

In [11]:
df['MA3'] = df['close'].rolling(3).mean()
df

,ts_code,close,MA3
trade_date,,,
2018-07-02,000001.SZ,8.61,NaN
2018-07-03,000001.SZ,8.67,NaN
2018-07-04,000001.SZ,8.61,8.630000
2018-07-05,000001.SZ,8.60,8.626667
2018-07-06,000001.SZ,8.66,8.623333
2018-07-09,000001.SZ,9.03,8.763333
2018-07-10,000001.SZ,8.98,8.890000
2018-07-11,000001.SZ,8.78,8.930000
2018-07-12,000001.SZ,8.88,8.880000


还有一个常用的窗口函数是expanding，每增加一行数据，窗口会相应的增大。比如，我们想计算某只股票每天的累计涨跌幅，就可以调用此函数。

In [14]:
df1 = df[['ts_code', 'pct_chg']]  # 列pct_chg单位是(%)
df1

,ts_code,pct_chg
0,000001.SZ,-0.23
1,000001.SZ,-0.11
2,000001.SZ,-1.69
3,000001.SZ,0.00
4,000001.SZ,2.78
5,000001.SZ,-2.23
6,000001.SZ,-0.55
7,000001.SZ,4.27
8,000001.SZ,0.70
9,000001.SZ,-0.12


In [17]:
df1['cum_pct_chg'] = df1['pct_chg'].expanding().sum()
df1

/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,ts_code,pct_chg,cum_pct_chg
0,000001.SZ,-0.23,-0.23
1,000001.SZ,-0.11,-0.34
2,000001.SZ,-1.69,-2.03
3,000001.SZ,0.00,-2.03
4,000001.SZ,2.78,0.75
5,000001.SZ,-2.23,-1.48
6,000001.SZ,-0.55,-2.03
7,000001.SZ,4.27,2.24
8,000001.SZ,0.70,2.94
9,000001.SZ,-0.12,2.82
